In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import os
import argparse

from models import VGG16Head, VGG16Tail, ResNet18Head, ResNet18Tail
import config
from watermark import Watermark
from tqdm import tqdm

import numpy as np

from bibdcalc import BIBD, BIBDParams

# parser = argparse.ArgumentParser()
# parser.add_argument('--model_name', help = 'Benchmark model structure.', choices = ['VGG16', 'ResNet18'])
# parser.add_argument('--dataset_name', help = 'Benchmark dataset used.', choices = ['CIFAR10', 'GTSRB'])
# parser.add_argument('-k', '--num_collusion', help = 'The number of attackers (k).', type = int, default = 2)
# parser.add_argument('-n', '--num_samples', help = 'The number of generated collusive samples.', type = int, default = 1000)
# parser.add_argument('--attack_name', help = 'Which black-box attack', choices = [ "Bandit", "NES", "HopSkipJump", "SignOPT", "SimBA-px"])
# parser.add_argument('--collusion_attack', help = 'collusion methods', choices = [ "mean", "max", "min", "median", "negative", "negative_prob"])
# parser.add_argument('-M', '--num_models', help = 'The number of models used.', type = int, default = 100)
# args = parser.parse_args()
class Args:
    def __init__(self):
        self.model_name = 'ResNet18'
        self.dataset_name = 'CIFAR10'
        self.num_collusion = 2
        self.num_samples = 100
        self.attack_name = 'NES'
        self.collusion_attack = 'mean'
        self.num_models = 10


args = Args()    

model_dir = f'saved_models/{args.model_name}-{args.dataset_name}'

total = 0
success_num = 0

prob = 0.8
a = np.load(f"saved_collusion_adv_examples/{args.model_name}-{args.dataset_name}/{args.num_collusion}_attackers/{args.attack_name}_{args.num_samples}_num_of_samples.npz", allow_pickle=True)

img = a['X'] # shape: n, 3, 32, 32
img_adv = a['X_attacked_k'] # shape: k, n, 3, 32, 32
label = a['y']
head_index = a['head']


adv_perturb = img_adv - img

if args.collusion_attack =='mean':
    collusion_perturb = np.mean(adv_perturb, axis=0) # (n, 3, 32, 32)
elif args.collusion_attack =='max':
    collusion_perturb = np.max(adv_perturb, axis=0) 
elif args.collusion_attack =='min':
    collusion_perturb = np.min(adv_perturb, axis=0) 
elif args.collusion_attack =='median':
    collusion_perturb = np.median(adv_perturb, axis=0) 
elif args.collusion_attack =='negative':
    collusion_perturb = np.max(adv_perturb, axis=0) + np.min(adv_perturb, axis=0) - np.median(adv_perturb, axis=0)
elif args.collusion_attack =='negative_prob':
    rand_mask = np.random.choice([1, 0], size=img.shape, p=[prob, 1-prob])
    collusion_perturb = np.max(adv_perturb, axis=0) * rand_mask + np.min(adv_perturb, axis=0) * (1 - rand_mask)
    collusion_perturb = collusion_perturb.astype(np.float32)

img_collusion = img + collusion_perturb

device = 'cuda' if torch.cuda.is_available() else 'cpu'

img =  torch.from_numpy(img).to(device)
img_adv = torch.from_numpy(img_adv).to(device)
img_collusion = torch.from_numpy(img_collusion).to(device)
label = torch.from_numpy(label).to(device)


dataset = eval(f'config.{args.dataset_name}()')
training_set, testing_set = dataset.training_set, dataset.testing_set
num_classes = dataset.num_classes
means, stds = dataset.means, dataset.stds


watermarks = []
for i in range(args.num_models):
    watermark = np.zeros((3, 32, 32))
    try:
        w_ = np.load(f'saved_models/ResNet18-CIFAR10/head_{i}/watermark.npy', allow_pickle=1)
    except:
        print('niubi')
    watermark[w_[:,0], w_[:,1], w_[:,2]] = 1
    watermarks.append(watermark)

watermarks = np.array(watermarks).astype(np.int32)

and_results = []
dic = {}
for i in range(watermarks.shape[0]):
    for j in range(i+1,watermarks.shape[0]):
        vec1 = watermarks[i]
        vec2 = watermarks[j]
        r = vec1 & vec2
        dic[len(and_results)] = (i, j)
        and_results.append(r)
        for v in and_results[:-1]:
            if (v == r).all():
                print("REPEAT")
                break
            
and_results = np.array(and_results)

success_count = 0
for i in tqdm(range(len(img_collusion))):
    # w_1 = watermarks[head_index[i][0]]
    # w_2 = watermarks[head_index[i][1]]

    # d_= (w_1 & w_2).reshape(3*32*32,).sum()
    # d_2 = 3*32*32 - d_
    # a = np.abs((collusion_perturb[i].reshape(3*32*32,) * (w_1 & w_2).reshape(3*32*32,))).sum() / d_
    # b = np.abs(collusion_perturb[i].reshape(3*32*32,) * (1- (w_1 & w_2).reshape(3*32*32,))).sum() / d_2
   
    # other_is_large = 0
    # idx_gt = 0
    # for k in range(45):
    #     d_3= and_results.reshape(-1, 3*32*32)[k].sum()
    #     b_ = np.abs(collusion_perturb[i].reshape(3*32*32,) * and_results.reshape(-1, 3*32*32)[k]).sum() / d_3
    #     if ((w_1 & w_2) == and_results[k]).all():
    #         idx_gt = k
    #     if (b_ > a):
    #         other_is_large += 1


    collusion_perturb = np.abs(collusion_perturb)
    
    denom = and_results.reshape(-1, 3*32*32).sum(axis=1)
    idx_ = ((collusion_perturb[i].reshape(3*32*32,) * and_results.reshape(-1, 3*32*32)).sum(axis=1)) / denom
    idx_ = idx_.argmin()
    head_pred = dic[idx_]

    if head_pred[0] == head_index[i][0] and head_pred[1] == head_index[i][1] or \
        head_pred[0] == head_index[i][1] and head_pred[1] == head_index[i][0]:
        success_count += 1

trace_acc = success_count / len(img_collusion)
print('the tracing accuracy is: ', trace_acc)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:00<00:00, 2188.96it/s]

the tracing accuracy is:  0.24


In [18]:
0.24 / 0.0009

266.6666666666667